In [74]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader


from customDatasets.audioDataset import AudioDataset


In [75]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_size),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded.view(-1, 1, 64, 313)

In [76]:
def train_model(model, train_dl, val_dl, criterion, optimizer, device, epochs=5):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        train_losses = []
        val_losses = []

        for inputs, labels in train_dl:
            model.train()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, inputs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        print(f'Epoch[{epoch + 1}/{epochs}], Loss: {np.average(train_losses): .4f}')
        
        for inputs, labels in val_dl:
            model.eval()
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, inputs)
                val_losses.append(loss.item())
            
        print(f'Epoch[{epoch + 1}/{epochs}], Val Loss: {np.average(val_losses): .4f}')

In [77]:
def set_seed(seed = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

CONFIG = {
    "seed": 42,
    "epochs": 10,
    "num_classes": 2,
    "learning_rate": 0.0001,
    "train_batch_size": 32,
    "val_batch_size": 128,
    "criterion": nn.MSELoss(),
    "device":
        torch.device(
            "cuda:0" if torch.cuda.is_available()
            else "mps" if torch.backends.mps.is_available()
            else "cpu"
        )
}

set_seed(CONFIG['seed'])

data_path = "./data/train/"

meta_train_df = pd.read_csv("./data/train.csv")

train_df = meta_train_df[['filename', 'is_normal', 'machine_id']]
train_ds = AudioDataset(train_df, data_path)

num_items = len(train_ds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train

train_ds, val_ds = random_split(train_ds, [num_train, num_val])


train_dl = DataLoader(train_ds, batch_size=CONFIG['train_batch_size'], shuffle=True)
val_dl = DataLoader(val_ds, batch_size=CONFIG['val_batch_size'], shuffle=False)

In [78]:
input_size = 64 * 313 # TODO: move to config
model = Autoencoder(input_size, encoding_dim=8)
model = model.to(CONFIG["device"])
optimizer = optim.Adam(model.parameters(), lr=CONFIG["learning_rate"])

In [79]:
train_model(model, train_dl, val_dl, CONFIG["criterion"], optimizer, CONFIG["device"], CONFIG["epochs"])

Epoch[1/10], Loss:  128.9110
Epoch[1/10], Val Loss:  87.8131
Epoch[2/10], Loss:  55.8615
Epoch[2/10], Val Loss:  31.1116
Epoch[3/10], Loss:  24.1535
Epoch[3/10], Val Loss:  19.7522
Epoch[4/10], Loss:  19.1814
Epoch[4/10], Val Loss:  18.1068
Epoch[5/10], Loss:  18.4197
Epoch[5/10], Val Loss:  17.9351
Epoch[6/10], Loss:  18.3052
Epoch[6/10], Val Loss:  17.9057
Epoch[7/10], Loss:  18.3420
Epoch[7/10], Val Loss:  17.8317
Epoch[8/10], Loss:  18.2358
Epoch[8/10], Val Loss:  17.9560
Epoch[9/10], Loss:  18.3305
Epoch[9/10], Val Loss:  17.7840
Epoch[10/10], Loss:  18.2390
Epoch[10/10], Val Loss:  17.8553
